In [ ]:
"""
File: image_stitcher.ipynb
Author: Clinton Reid
Description: stitches multiple fields of view into one image considering overlap; aligns images across imaging rounds
             assumes this notebook is in the same folder as the imaging round folders.
"""

In [2]:
# built-in imports
import os

# libraries
import cv2 as cv
import numpy as np
import pandas as pd

In [3]:
current_folder = os.getcwd()

In [4]:
locations_csv_path = os.path.join(current_folder, 'image_locations.csv')
image_locations = pd.read_csv(locations_csv_path)

overlap = 0.15

In [6]:
def vertical_stitch_images(images_to_stitch, overlap):
    """
    Returns the result of stitching multiple images vertically with same associated overlap.

        Parameters:
            images_to_stitch (list of numpy arrays): same-width images in the order they are to be joined in. 
            Earlier images in the list are above later ones.

            overlap (float): decimal representing the percentage overlap between images

        Returns:
            vertical_stitched (numpy array): image representing the stitching of images in images_to_stitch
    """
    overlap_height = int(images_to_stitch[0].shape[0] * overlap)

    vertical_stitched = images_to_stitch[0] # initialize the stitched image as the first

    # iteratively stitch each remaining image to the previously stitched image
    for i in range(1, len(images_to_stitch)): 
        non_overlap_region = images_to_stitch[i][overlap_height:, :] # region that doesn't overlap with previous stitched image

        vertical_stitched = np.vstack([vertical_stitched[:-overlap_height, :], non_overlap_region])

    return vertical_stitched

def horizontal_stitch_images(images_to_stitch, overlap):
    """
    Returns the result of stitching multiple images horizontally with same associated overlap.

        Parameters:
            images_to_stitch (list of numpy arrays): same-height images in the order they are to be joined in.
            Earlier images in the list are to the left of later ones.
            
            overlap (float): decimal representing the percentage overlap between images

        Returns:
            horizontal_stitched (numpy array): image representing the stitching of images in images_to_stitch
    """
    overlap_width = int(images_to_stitch[0].shape[1] * overlap)

    horizontal_stitched = images_to_stitch[0]
    for i in range(1, len(images_to_stitch)):
        non_overlap_region = images_to_stitch[i][:, overlap_width:]
        new_image = np.hstack([horizontal_stitched[:, :-overlap_width], non_overlap_region])
        horizontal_stitched = new_image

    return horizontal_stitched


In [7]:
def create_fov_grid(image_locations, current_folder, round_folder):
    """
    Returns a nested array of images corresponding to their capture positions on the slide.

        Parameters:
            image_locations (pandas.DataFrame): table containing (x, y) positional info for each field on slide
                Index:
                    RangeIndex
                Columns:
                    Name: FieldID, dtype: int64
                    Name: X, dtype: float64
                    Name: Y, dtype: float64

            current_folder (str): path to current working directory

            round_folder (str): name of folder containing images for a particular imaging round

        Returns:
            image_fov_grid (nested list of numpy arrays): list of images representing their row and column positions on slide
    """
    
    # fields of view sorted by Y coordinate first since we are building this grid row-wise
    sorted_image_locations = image_locations.sort_values(by=['Y', 'X'])
    
    image_fov_grid = []
    current_y = None
    current_img_row = None

    for idx, row in sorted_image_locations.iterrows():
        # loading images from bottom left corner to top right corner

        field_id = str(int(row['FieldID'])).zfill(2) # leading 0's necessary for f01-f09
        filename = f'r01c04f{field_id}p02-ch1sk1fk1fl1.tiff'

        image_path = os.path.join(current_folder, round_folder, 'Images', filename)
        image = cv.imread(image_path)

        # in order to build row-wise, keep track of current value of Y
        if row['Y'] != current_y:
            # for every new value of Y, build a new image row
            current_y = row['Y']
            current_img_row = [image]
            image_fov_grid.append(current_img_row)
        else:
            # otherwise, continue building the current row
            current_img_row.append(image)
    
    return image_fov_grid

In [8]:
round_nums = [1,2]

for round in round_nums:
    round_folder = 'Round' + str(round)
    image_fov_grid = create_fov_grid(image_locations, current_folder, round_folder)
    horizontal_panoramas = []

    for image_row in image_fov_grid:
        # stitch together all images in each row into individual panoramas
        horizontal_panorama = horizontal_stitch_images(image_row, overlap)
        if horizontal_panorama is not None:
            horizontal_panoramas.append(horizontal_panorama)
        
    horizontal_panoramas = horizontal_panoramas[::-1] # avoids upside down stitching

    full_image = vertical_stitch_images(horizontal_panoramas, overlap) # stitch panoramas into complete image
    cv.imwrite(os.path.join(current_folder, round_folder, 'Images', 'stitched.tiff'), full_image)


In [9]:
all_files = os.listdir(current_folder)
rounds = sorted([file for file in all_files if 'Round' in file]) # folders containing imaging data from each round

image_list = [cv.imread(os.path.join(current_folder, round, 'Images', 'stitched.tiff'), cv.IMREAD_GRAYSCALE) for round in rounds]

In [10]:
def pad_image(image):
    """
    Returns an image with a 5% black padding around it.

        Parameters:
            image (numpy array)

        Returns:
            padded_image (numpy array): padded image
    """
    height, width = image.shape
    padded_image = cv.copyMakeBorder(image, int(0.05*height), int(0.05*height), int(0.05*width), int(0.05*width), cv.BORDER_CONSTANT, value=(0,0,0))
    return padded_image

def align_and_save_images(image_list, current_folder):
    """
    Aligns images to a single reference image and saves all of them.

        Parameters:
            image_list: list of images (as arrays) to be aligned. it is expected that image_list[0] be the reference image for all alignments.
                        it is also expected that all images in image_list are the same size or larger than the reference.

            current_folder: path to current working directory

        Returns:
            Saves aligned images as tiffs together in 'align' folder in current working directory. reference will be saved as aligned_1.tiff
            All output images are of the same dimensions as reference image.
    """
    ref_image = image_list[0]
    align_folder = os.path.join(current_folder, 'align')
    if not os.path.exists(align_folder):
        os.makedirs(align_folder)

    cv.imwrite(f"{align_folder}/aligned_1.tiff", ref_image)
    height, width = ref_image.shape

    for i in range(1, len(image_list)):
        print('template matching...')
        padded_image = pad_image(image_list[i]) # creates a padded version of image to be aligned

        # template matching slides the reference image over the padded image and returns a grayscale image corresponding
        # to how well the reference matches the padded image at that position in terms of cross-correlation (in this case)

        res = cv.matchTemplate(padded_image, ref_image, cv.TM_CCOEFF_NORMED)
        
        # identify the position of the reference image's top-left corner corresponding to maximum correlation
        # it is this position that will be the top-left ofthe newly aligned image

        _, _, _, max_loc = cv.minMaxLoc(res) 
        x, y = max_loc

        aligned_img = padded_image[y: y + height, x: x + width]

        cv.imwrite(f"{align_folder}/aligned_{i+1}.tiff", aligned_img)

    return True


In [11]:
align_and_save_images(image_list, current_folder)

template matching...


True